# 📰 Fake News Generator & Detector

In [ ]:

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import gradio as gr
import torch
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.calibration import CalibratedClassifierCV


In [ ]:

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


In [ ]:

def generate_news(prompt, max_length=100):
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(inputs, max_length=max_length, do_sample=True, top_k=50, top_p=0.95, temperature=0.9)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:

data = {
    'text': [
        "Government launches a new scheme to support farmers.",
        "Aliens have landed in the Pacific Ocean.",
        "New study shows benefits of drinking green tea.",
        "Actor claims to be from another galaxy.",
        "Election results declared for 2025 polls.",
        "Scientists find cure for aging using moon dust.",
        "PM announces new education policy focused on digital learning.",
        "Vampires spotted in downtown London."
    ],
    'label': [0, 1, 0, 1, 0, 1, 0, 1]
}

df = pd.DataFrame(data)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text'])
y = df['label']

model_nb = CalibratedClassifierCV(estimator=MultinomialNB(), cv=3)
model_nb.fit(X, y)

def detect_fake_news_confidence(text):
    features = vectorizer.transform([text])
    prediction = model_nb.predict(features)[0]
    proba = model_nb.predict_proba(features)[0][prediction]
    label = "🟢 REAL" if prediction == 0 else "🔴 FAKE"
    return f"{label} ({proba*100:.2f}% confidence)"


In [ ]:

with gr.Blocks(theme=gr.themes.Base()) as app:
    gr.Markdown("## 📰 Fake News Generator & Detector")
    gr.Markdown("Generate fake news using GPT-2 and detect it using a confidence-calibrated classifier.")

    with gr.Row():
        with gr.Column(scale=1):
            prompt = gr.Textbox(label="Enter News Prompt")
            gen_btn = gr.Button("Submit")
            clear_btn = gr.Button("Clear")
        with gr.Column(scale=2):
            output_news = gr.Textbox(label="Generated Headline")
            detection_result = gr.Textbox(label="Detection Result")

    def handle_submission(prompt):
        generated = generate_news(prompt)
        detection = detect_fake_news_confidence(generated)
        return generated, detection

    gen_btn.click(fn=handle_submission, inputs=prompt, outputs=[output_news, detection_result])
    clear_btn.click(fn=lambda: ("", ""), inputs=None, outputs=[output_news, detection_result])

app.launch()
